In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%cd DeepLearning_Financial

## EXTERNAL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import yfinance
from pandas import Series
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, PredefinedSplit
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from IPython.display import display
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import time
import os
import random 
from sklearn.datasets import make_regression
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from torch.nn.modules.loss import MSELoss
import tensorflow as tf
from tensorflow import keras
from skorch.dataset import CVSplit
from skorch import callbacks
import pickle
from sklearn.model_selection import train_test_split
from functools import partial
import skorch
import pywt
from sklearn import preprocessing
import joblib


##INTERNAL
from models import Autoencoder, waveletSmooth, SequenceDouble, SequenceDoubleAtt, SequenceAtt
from utils import prepare_data_lstm, ExampleDataset, save_checkpoint, evaluate_lstm, backtest
from testing import *
from sa_tools import *
from test_models import *



#creazione dataset
# indices = ['^DJI', '^GSPC', '^HSI', '^N225', 'ASHR','^NSEI']
indices = ['^GSPC']
feature_sets = ['open', 'ohlcv', 'ext']
start_date="2000-01-01"
end_date="2018-12-31"

# datasets=get_datasets(indices = indices, feature_sets = feature_sets, end_date=end_date, start_date=start_date)
# save(datasets, 'datasets')
datasets = load('^GSCP')
# tss = TimeSeriesSplit(4)
# tss_split = CVSplit(cv=tss, stratified=False, random_state=None)


for index in indices:
    X=datasets[index]["target"]
    
    tss = TimeSeriesSplit(4)
    max_train_size = len(list(tss.split(X))[0][0])
    tss = TimeSeriesSplit(4, max_train_size=max_train_size)
    tss_split = CVSplit(cv=tss, stratified=False, random_state=None)
    
    for i, (train, val) in enumerate(tss.split(X), start=1):
        print("FOLD {}".format(i))
        train_dates = X.index[train]
        val_dates = X.index[val]
        print("Training set da {} a {}".format(train_dates.min(), train_dates.max()))
        print("Validation set da {} a {}".format(val_dates.min(), val_dates.max()))
        print(X.index.min())
        print(X.index.max())
    for i, (train_dates, val_dates) in enumerate(tss.split(datasets[index]["target"]), start=1):
        for model_name in get_model_names():
            
            # get the model
            save_name = "{}_{}_{}_{}.pkl".format(model_name, index, datasets[index]["target"].index[train_dates].min().date(), datasets[index]["target"].index[train_dates].max().date())
            print(save_name)
            model, feature_set = get_model(model_name,  save_name)
                
            #get the dataset
            sa=None
            ld=False
            fset= feature_set
            if(feature_set is 'ext_sa'):
                fset='ext'
                sa = "sa_{}_{}_{}".format(index, datasets[index]["target"][train_dates].index.min().date(), datasets[index]["target"][train_dates].index.max().date())
                sa_fname = sa + ".pkl"
                if(os.path.isfile(sa_fname)):
                    ld=True
            x, y, x_train, x_val, x_scaler, y_train, y_val, y_scaler = get_dataset_train(datasets=datasets, feature_set=fset, train_dates=train_dates, val_dates=val_dates, index=index, sa=sa, ld=ld)
            
            
            #prepare dataset for training
            n_days=5

            x_batch = days_group(x, n_days=n_days)
            y_batch = y[n_days:]
            serie = x_batch
            train_dates_s = train_dates[n_days:].copy()
            val_dates_s = val_dates.copy()

            splitted = np.split(serie, [len(train_dates_s)])
            l1, l2 = len(splitted[0]), len(splitted[1])
            train_s = PredefinedSplit(np.concatenate((np.ones(l1)*-1,np.ones(l2))))
            train_split =  CVSplit(cv=train_s, stratified=False, random_state=None)  
            train_dates_s, val_dates_s = list(train_s.split(serie))[0]
            
            model.train_split = train_split
            model.fit(x_batch,y_batch)
    

/data/home/dsvm_server_admin/notebooks/fastai/tesi/DeepLearning_Financial
FOLD 1
Training set da 2000-09-19 00:00:00 a 2004-05-18 00:00:00
Validation set da 2004-05-19 00:00:00 a 2008-01-12 00:00:00
2000-09-19 00:00:00
2018-12-27 00:00:00
FOLD 2
Training set da 2004-05-15 00:00:00 a 2008-01-12 00:00:00
Validation set da 2008-01-13 00:00:00 a 2011-09-07 00:00:00
2000-09-19 00:00:00
2018-12-27 00:00:00
FOLD 3
Training set da 2008-01-09 00:00:00 a 2011-09-07 00:00:00
Validation set da 2011-09-08 00:00:00 a 2015-05-03 00:00:00
2000-09-19 00:00:00
2018-12-27 00:00:00
FOLD 4
Training set da 2011-09-04 00:00:00 a 2015-05-03 00:00:00
Validation set da 2015-05-04 00:00:00 a 2018-12-27 00:00:00
2000-09-19 00:00:00
2018-12-27 00:00:00
lstm_sa_^GSPC_2000-09-19_2004-05-18.pkl
  epoch    neg_mean_absolute_error       r2    train_loss    valid_loss    cp     dur
-------  -------------------------  -------  ------------  ------------  ----  ------
      1                    -1.3908  -2.5641        1.0

/data/anaconda/envs/fastai/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


      2                    -1.4024  -2.6081        1.0186        2.7144        2.2816
      3                    -1.4072  -2.6262        1.0123        2.7280        1.7384
      4                    -1.4098  -2.6364        1.0079        2.7356        2.2214
      5                    -1.4114  -2.6425        1.0048        2.7402        1.5448
      6                    -1.4125  -2.6468        1.0025        2.7435        1.5456
      7                    -1.4133  -2.6497        1.0007        2.7456        1.6991
      8                    -1.4139  -2.6520        0.9993        2.7474        1.5616
      9                    -1.4143  -2.6536        0.9981        2.7486        1.8465
     10                    -1.4147  -2.6549        0.9972        2.7496        1.7556
     11                    -1.4150  -2.6561        0.9964        2.7505        2.0910
     12                    -1.4153  -2.6574        0.9957        2.7514        2.0470
     13                    -1.4156  -2.6585        0.9